In [55]:
from langgraph.graph import MessagesState
from typing import List,  Dict, TypedDict
class SupervisorState(MessagesState):
    """State for the multi-agent research system."""
    
    next_agent : str 
    task_complete : bool 
    current_task : str
    papers_raw : List[Dict]
    ranked_paper : List[Dict]
    research_gaps : str 
    analysis : str 
    refined_focus: str    
 
    
    

In [56]:
from langchain.chat_models import init_chat_model
import os

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
llm = init_chat_model("groq:openai/gpt-oss-120b")

In [57]:
import arxiv
from typing import List, Dict
from langchain_core.tools import tool

@tool
def scraper_tool(query: str, max_results: int = 3, max_chars: int = 3000) -> Dict:
    """
    Scraper tool: collects recent research papers from arXiv based on the query.

    Args:
        query (str): The search query/topic.
        max_results (int): Maximum number of papers to retrieve.
        max_chars (int): Maximum number of characters from the abstract.

    Returns:
        Dict: Contains a list of papers with basic metadata.
    """
    client = arxiv.Client()
    search = arxiv.Search(
        query=query,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.SubmittedDate
    )

    papers: List[Dict] = []

    for result in client.results(search):  # use Client.results() as recommended
        paper_summary = result.summary[:max_chars]
        papers.append({
            "title": result.title,
            "authors": [author.name for author in result.authors],
            "abstract": paper_summary,
            "citations": 0,  # placeholder, arXiv does not provide citations
            "year": result.published.year,
            "source": "ArXiv"
        })

    return {"papers": papers}


In [58]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage


from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def create_supervisor_chain():
    """Creates the supervisor decision chain"""
    
    supervisor_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a supervisor managing a team of agents:
        
1. Scraper - Collects research papers from arXiv  
2. Ranker - Ranks papers by relevance  
3. Gap_Finder - Identifies research gaps  
4. Feedback - Refines the research focus  

Based on the current state and conversation, decide which agent should work next.
If the task is complete, respond with 'DONE'.

Current state:
- Has raw papers: {has_papers}
- Has ranked papers: {has_ranked}
- Has research gaps: {has_gaps}
- Has refined focus: {has_focus}

Respond with ONLY the agent name (scraper/ranker/gap_finder/feedback) or 'DONE'."""),
        ("human", "{task}")
    ])
    
    return supervisor_prompt | llm | StrOutputParser()




def build_ranker_prompt(papers, topic: str) -> str:
    """
    Build a ranking prompt for the LLM.
    """
    paper_texts = "\n\n".join(
        [f"{i+1}. Title: {p['title']}\nAbstract: {p['abstract'][:300]}..."
         for i, p in enumerate(papers)]
    )

    prompt = f"""
You are an expert researcher in {topic}.
Here are some papers collected on this topic:

{paper_texts}

Your tasks:
1. Rank these papers from most relevant to least relevant for the topic: "{topic}".
2. For each paper, explain briefly why you placed it at that rank.
3. Suggest the top paper that should be read first.
"""

    return prompt


def build_gap_finder_prompt(papers, topic: str) -> str:
    """    Build a prompt for identifying gaps in research.
    """
    paper_texts = "\n\n".join(
        [f"Title: {p['title']}\nAbstract: {p['abstract'][:400]}..."
         for p in papers]
    )

    return f"""
You are an expert analyst. Your job is to identify gaps in current research.

Topic: {topic}

Here are the abstracts of recent papers:
{paper_texts}

Your tasks:
1. Summarize the main areas that these papers cover.
2. Identify important gaps or underexplored areas in this topic.
3. Suggest 2–3 specific research questions that remain unanswered.
"""


def build_feedback_prompt(gaps: str, topic: str) -> str:
    """
    Build a prompt to interact with the user for refining the search.
    """
    return f"""
You are an assistant helping refine a research direction.

Topic: {topic}

Based on current analysis, here are the identified gaps:
{gaps}

Please ask the user clarifying questions to refine the search focus.
For example:
- Which sub-area is most interesting?
- Should we prioritize newer or more cited works?
- Do we want theory-focused or application-focused research?
"""


In [59]:


from langchain_core.messages import HumanMessage
import time
from langgraph.graph import END
from langchain.chat_models import init_chat_model
import os




from langchain_core.messages import AIMessage

from langchain_core.messages import AIMessage
from langgraph.graph import END

def supervisor_agent(state: SupervisorState) -> dict:
    """Supervisor decides the next agent using Groq LLM and current state."""

    messages = state.get("messages", [])
    task = state.get("current_task", messages[-1].content if messages else "No task")

    # Check what has been done
    has_papers = bool(state.get("papers_raw"))
    has_ranked = bool(state.get("ranked_paper"))
    has_gaps = bool(state.get("research_gaps"))
    has_focus = bool(state.get("refined_focus"))

    # Invoke LLM chain to get suggested next agent
    chain = create_supervisor_chain()
    decision = chain.invoke({
        "task": task,
        "has_papers": has_papers,
        "has_ranked": has_ranked,
        "has_gaps": has_gaps,
        "has_focus": has_focus
    })

    # Normalize and interpret decision
    decision_text = decision.strip().lower()  # remove whitespace/newlines
    print("LLM suggested:", decision_text)

    # Determine next agent
    if "done" in decision_text or has_focus:
        next_agent = END
        supervisor_msg = "✅ Supervisor: All tasks complete! Great work team."
    elif "scraper" in decision_text or not has_papers:
        next_agent = "scraper"
        supervisor_msg = "📋 Supervisor: Assigning task to Scraper agent..."
    elif "ranker" in decision_text or (has_papers and not has_ranked):
        next_agent = "ranker"
        supervisor_msg = "📋 Supervisor: Papers ready. Assigning task to Ranker agent..."
    elif "gap_finder" in decision_text or (has_ranked and not has_gaps):
        next_agent = "gap_finder"
        supervisor_msg = "📋 Supervisor: Analysis done. Assigning task to Gap Finder agent..."
    elif "feedback" in decision_text or (has_gaps and not has_focus):
        next_agent = "feedback"
        supervisor_msg = "📋 Supervisor: Gaps identified. Assigning task to Feedback agent..."
    else:
        next_agent = END
        supervisor_msg = "✅ Supervisor: Task seems complete."

    return {
        "messages": [AIMessage(content=supervisor_msg)],
        "next_agent": next_agent,
        "current_task": task
    }




def scraper_agent(state: SupervisorState) -> Dict:
    """Scrapes research papers using the scraper_tool."""
    if state.get("papers_raw"):  
        return {
            "messages": [AIMessage(content="Scraper agent: papers already scraped, skipping.")],
            "papers_raw": state["papers_raw"],
            "next_agent": "supervisor"
        }

    task = state.get("current_task", "")
    if not task:
        return {
            "messages": [AIMessage(content="Scraper agent: no current task provided.")],
            "papers_raw": [],
            "next_agent": "supervisor"
        }

    # Directly call scraper_tool (instead of LLM hallucinating a call)
    result = scraper_tool.invoke({"query": task, "max_results": 5})
    papers = result.get("papers", [])

    agent_message = f"Scraper agent: retrieved {len(papers)} papers for task '{task}'."

    return {
        "messages": [AIMessage(content=agent_message)],
        "papers_raw": papers,
        "next_agent": "supervisor"
    }
    



def ranker_agent(state: SupervisorState) -> Dict:
    """Ranks scraped papers by relevance."""
    if not state.get("papers_raw"):
        return {
            "messages": [AIMessage(content="Ranker agent: no papers available to rank.")],
            "ranked_paper": [],
            "analysis": "",
            "next_agent": "supervisor"
        }

    prompt = build_ranker_prompt(state.get("papers_raw"), state.get("current_task"))
    time.sleep(1)
    response = llm.invoke([HumanMessage(content=prompt)])
    summary = response.content[:3000]

    agent_message = (
        f"Ranker agent: I ranked {len(state.get('papers_raw', []))} papers "
        f"for task '{state.get('current_task')}'. Summary: {summary}"
    )

    return {
        "messages": [AIMessage(content=agent_message)],
        "ranked_paper": state.get("papers_raw"),  # placeholder, could be improved with real ranking
        "analysis": summary,
        "next_agent": "supervisor"
    }



def gap_finder_agent(state: SupervisorState) -> Dict:
    """Identifies research gaps in the ranked papers."""
    if not state.get("ranked_paper"):
        return {
            "messages": [AIMessage(content="Gap Finder agent: no ranked papers available.")],
            "research_gaps": "",
            "next_agent": "feedback"
        }

    prompt = build_gap_finder_prompt(state.get("ranked_paper"), state.get("current_task"))
    time.sleep(1)
    response = llm.invoke([HumanMessage(content=prompt)])
    summary = response.content[:3000]

    agent_message = f"Gap Finder agent: I identified gaps for task '{state.get('current_task')}'. Summary: {summary}"
    return {
        "messages": [AIMessage(content=agent_message)],
        "research_gaps": summary,
        "next_agent": "supervisor"
    }


def feedback_agent(state: SupervisorState) -> Dict:
    """Refines research focus."""
    if not state.get("research_gaps"):
        state["refined_focus"] = ""
        state["next_agent"] = END
        return state

    prompt = build_feedback_prompt(state.get("research_gaps"), state.get("current_task"))
    time.sleep(1)
    response = llm.invoke([HumanMessage(content=prompt)])
    summary = response.content[:3000]
    
    state["refined_focus"] = summary
    state["next_agent"] = END
    
    agent_message = f" Feedback agent: I completed feedback for {state.get("current_task")}. Here is the summary{summary} "
    
    return{
        "messages" : [AIMessage(content=agent_message)],
        "refined_focus" : summary,
        "next_agent" : "supervisor"
    }

In [ ]:
from typing import Dict
from langgraph.graph import StateGraph, END

def router(state: Dict) -> str:
    """Router follows supervisor's decision, normalizing output."""
    if state.get("task_complete"):
        return END
    return state.get("next_agent", "").strip().lower()


def build_workflow(llm):
    """Builds the multi-agent research workflow."""
    workflow = StateGraph(SupervisorState)

    # Add agents as nodes
    workflow.add_node("supervisor", supervisor_agent)
    workflow.add_node("scraper", scraper_agent)
    workflow.add_node("ranker", ranker_agent)
    workflow.add_node("gap_finder", gap_finder_agent)
    workflow.add_node("feedback", feedback_agent)

    # Set entry point
    workflow.set_entry_point("supervisor")

    # Add conditional routing from supervisor
    workflow.add_conditional_edges(
        "supervisor",
        router,
        {
            "supervisor": "supervisor",
            "scraper": "scraper",
            "ranker": "ranker",
            "gap_finder": "gap_finder",
            "feedback": "feedback",
            END: END,
        }
    )

    # Add linear routing for other agents → always back to supervisor
    workflow.add_edge("scraper", "supervisor")
    workflow.add_edge("ranker", "supervisor")
    workflow.add_edge("gap_finder", "supervisor")
    workflow.add_edge("feedback", "supervisor")

    return workflow.compile()


if __name__ == "__main__":
    llm = init_chat_model("groq:gemma2-9b-it")
    graph = build_workflow(llm)

    # Initial state
    state = {
        "next_agent": "supervisor",
        "task_complete": False,
        "current_task": "Quantum Machine Learning",
        "papers_raw": [],
        "ranked_paper": [],
        "research_gaps": "",
        "analysis": "",
        "refined_focus": "",
    }

    # Run compiled workflow
for step in graph.stream(state):
    for agent, output in step.items():
        print(f"Agent: {agent}")
        messages = output.get("messages", [])
        for msg in messages:
            # Here is where .content is used
            print("Message content:", msg.content)
        print("---")


LLM suggested: scraper
Agent: supervisor
Message content: 📋 Supervisor: Assigning task to Scraper agent...
---
Agent: scraper
Message content: Scraper agent: retrieved 5 papers for task 'Quantum Machine Learning'.
---
LLM suggested: ranker
Agent: supervisor
Message content: 📋 Supervisor: Papers ready. Assigning task to Ranker agent...
---
Agent: ranker
Message content: Ranker agent: I ranked 5 papers for task 'Quantum Machine Learning'. Summary: Here's a ranking of the papers based on relevance to "Quantum Machine Learning", along with explanations:

**1. High-capacity associative memory in a quantum-optical spin glass**

* **Relevance:** This paper directly explores a quantum mechanical system (spin glass) and applies it to a classic machine learning problem (associative memory). It delves into the potential of quantum phenomena for enhancing memory capabilities. This is a core area of research in Quantum Machine Learning.

**2.  Dynamic Relational Priming Improves Transformer in Mult